In [252]:
import os
import string,sys
from jinja2 import Environment, FileSystemLoader
import numpy as np

In [253]:
opt_f='bromide.log'
not_opt_f='coumarin.log'
with open(opt_f, 'r') as f:
    lines = f.readlines()

In [254]:
# Check if optimization was succesfull 
opt = False
for line in lines:
    if  "**** THE GEOMETRY SEARCH IS NOT CONVERGED! ****" in line:
        opt = True

In [255]:
# If optimization is successfull 
if not opt:
    data = [] 
    for num, line in enumerate(lines):
        if "TOTAL NUMBER OF ATOMS" in line:
            n_atoms = int(line.split()[5])
        if "EQUILIBRIUM" in line:
            start_coord_index = num + 4
            end_coord_index = start_coord_index + n_atoms
            for i in range(start_coord_index, end_coord_index):
                data.append(lines[i].split())
                
    # store opt coordinates into a list
    atomic_number = []
    element_symbol = []
    x_coord = []
    y_coord = []
    z_coord = []
    for line in data:
        element_symbol.append(line[0])
        atomic_number.append(line[1])
        x_coord.append(line[2])
        y_coord.append(line[3])
        z_coord.append(line[4])
    opt_coord = (element_symbol, x_coord, y_coord, z_coord)
    result = np.transpose(np.array(opt_coord))

In [256]:
# Option 1
if not opt:
    file_loader=FileSystemLoader("../templates")
    environment = Environment(loader=file_loader)
    template = environment.get_template("structure.xyz")

    MOLNAME="bromide"
    content = template.render(N_ATOMS=n_atoms, MOLNAME=MOLNAME, COORDINATES=result)
    print(content)

In [257]:
# Option 2    

# opt_xyz = open(MOLNAME+'.xyz','w')
# opt_xyz.write(str(n_atoms) +'\n')
# opt_xyz.write(MOLNAME +'\n')
if not opt:
        print(MOLNAME +'\n')
        print(str(n_atoms) +'\n')
        for i in range(n_atoms):
                line="{0:<4s}".format(element_symbol[i]) + ' ' 
                line+="{0:>16.10f}".format(float(x_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(y_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(z_coord[i])) + ' ' + '\n'
                print(line)
#         opt_xyz.write(line)
# opt_xyz.close()

In [258]:
# If optimization is NOT successfull 

if  opt:
	file_loader=FileSystemLoader("../templates")
	environment = Environment(loader=file_loader)
	template = environment.get_template("gamess_input.inp")
	data = []
	for num, line in enumerate(lines):
		if "TOTAL NUMBER OF ATOMS" in line:
			n_atoms = int(line.split()[5])
		if "ALWAYS THE LAST POINT COMPUTED!" in line:
			start_coord_index = num + 4
			end_coord_index = start_coord_index + n_atoms
			for i in range(start_coord_index, end_coord_index):
				data.append(lines[i])

	SCFTYP='RHF'
	MULT=1 
	OPTTOL=0.0005
	NSTEP=100 
	GBASIS='N31' 
	NGAUSS=6
	MOLNAME="bromide"
	content = template.render(SCFTYP=SCFTYP, MULT=MULT, OPTTOL=OPTTOL,
			NSTEP=NSTEP, GBASIS=GBASIS, NGAUSS=NGAUSS, MOLNAME=MOLNAME, COORDINATES=data)
	print(content)

 $CONTRL SCFTYP=RHF MULT=1 RUNTYP=OPTIMIZE $END
 $STATPT OPTTOL=0.0005 NSTEP=100 $END
 $BASIS  GBASIS=N31 NGAUSS=6 $END

$DATA
C1
 O           8.0  -3.1653408462   0.6356877423  -0.0159611523
 C           6.0  -2.0638705169   0.1433353987  -0.0422337969
 O           8.0  -0.9641934909   0.9716032877  -0.0674019256
 C           6.0  -1.7750007318  -1.2871835302  -0.0521999738
 C           6.0   0.3390418371   0.5322400519  -0.0396322582
 C           6.0  -0.5182488792  -1.7406343568  -0.0355879939
 C           6.0   0.6118633369  -0.8297790609  -0.0099302446
 C           6.0   1.3447358463   1.4824675604  -0.0359253715
 C           6.0   1.9458791868  -1.2400669489   0.0462841369
 C           6.0   2.6580408122   1.0544885021   0.0182395050
 C           6.0   2.9623278007  -0.3066132702   0.0659246829
 H           1.0  -2.6260430679  -1.9339135387  -0.0683227935
 H           1.0  -0.3226127769  -2.7958011481  -0.0418992733
 H           1.0   1.0829097033   2.5196494116  -0.0709490648
 H